#### WARNING: THIS NOTEBOOK TAKES HOURS TO RUN TO COMPLETION. DO NOT RUN UNLESS YOU MUST.

## Feature Selection on Tree-Based Models

In [1]:
import time
import json
from tqdm import tqdm
from itertools import compress
from collections import defaultdict
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SequentialFeatureSelector # requires sklearn 0.24 and above
from features import feature_pipeline
from catboost import CatBoostClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

### Data Loading

In [49]:
with open("../data/train_features.json", "r", encoding="utf-8") as f:
    train = pd.DataFrame(json.load(f))
with open("../data/val_features.json", "r", encoding="utf-8") as f:
    val = pd.DataFrame(json.load(f))
with open("../data/test_features.json", "r", encoding="utf-8") as f:
    test = pd.DataFrame(json.load(f))

X_train = train.drop(["preprocessed_text", "level"], axis=1)
y_train = train["level"].tolist()
X_val = val.drop(["preprocessed_text", "level"], axis=1)
y_val = val["level"].tolist()
X_test = test.drop(["preprocessed_text", "level"], axis=1)
y_test = test["level"].tolist()

y_train_binary = []
for lvl in train["level"].tolist():
    y_train_binary.append(lvl[0])
y_val_binary = []
for lvl in val["level"].tolist():
    y_val_binary.append(lvl[0])
y_test_binary = []
for lvl in test["level"].tolist():
    y_test_binary.append(lvl[0])

In [3]:
print(y_val)

['B', 'A1', 'B', 'A2', 'A2', 'A1', 'A1', 'A1', 'B', 'A2', 'B', 'B', 'B', 'B', 'A2', 'A1', 'B', 'B', 'A1', 'A1', 'B', 'B', 'B', 'A1', 'A2', 'A2', 'A1', 'A1', 'B', 'A1', 'B', 'B']


In [4]:
print(y_val_binary)

['B', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'B']


In [5]:
feat_names = list(X_train.columns)
print(f"All features:\n{feat_names}\n\nNumber of features: {len(feat_names)}")

scoring = accuracy_score

All features:
['total_tokens', 'total_tokens_w/o_stopwords', 'avg_sent_length', 'proportion_of_A_level_tokens', 'proportion_of_A_level_types', 'num_connectives', 'logical_operator_density', 'pronoun_density', 'type_token_ratio', 'avg_rank_of_lemmas_in_freq_list', 'fernandez_huerta_score', 'syllables_per_sentence', 'avg_degree_of_abstraction', 'min_degree_of_abstraction', 'avg_ambiguation_all_words', 'avg_ambiguation_content_words', 'noun_phrase_density', 'avg_parse_tree_depth', 'Fut', 'Imp', 'Past', 'Pres', 'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'EOL', 'SPACE', 'CONTENT', 'FUNCTION']

Number of features: 44


In [6]:
X_train.head()

,total_tokens,total_tokens_w/o_stopwords,avg_sent_length,proportion_of_A_level_tokens,proportion_of_A_level_types,num_connectives,logical_operator_density,pronoun_density,type_token_ratio,avg_rank_of_lemmas_in_freq_list,...,PROPN,PUNCT,SCONJ,SYM,VERB,X,EOL,SPACE,CONTENT,FUNCTION
0,2585,1118,35.410959,0.339893,0.182550,18,0.050386,0.047407,0.360155,684.322631,...,0.011605,0.109865,0.032882,0.000000,0.078530,0.0,0.0,0.000000,0.635376,0.364624
1,1539,622,14.941748,0.326367,0.214612,18,0.034970,0.067268,0.388564,629.897336,...,0.010396,0.152697,0.038337,0.005198,0.107862,0.0,0.0,0.000000,0.621914,0.378086
2,159,70,22.714286,0.557143,0.395833,7,0.039216,0.143885,0.578616,611.383648,...,0.012579,0.075472,0.025157,0.000000,0.169811,0.0,0.0,0.000000,0.673469,0.326531
3,291,117,22.384615,0.307692,0.250000,9,0.043011,0.024648,0.580756,547.487973,...,0.013746,0.189003,0.034364,0.000000,0.089347,0.0,0.0,0.003436,0.629787,0.370213
4,401,193,21.105263,0.295337,0.231250,5,0.046997,0.028205,0.551122,528.177057,...,0.019950,0.164589,0.022444,0.000000,0.109726,0.0,0.0,0.004988,0.654655,0.345345


### Set up Functions

In [7]:
def cross_validate(pipeline, X_train, y_train, X_val, y_val):
    """"""
    start = time.time()
    pipeline.fit(X_train, y_train)
    train_time = time.time() - start

    train_score = pipeline.score(X_train, y=y_train)

    start = time.time()
    val_score = pipeline.score(X_val, y=y_val)
    val_pred_time = time.time() - start

    return {
        "Training_score": train_score,
        "Validation_score": val_score,
        "Training_time": train_time,
        "Prediction_time": val_pred_time,
    }

In [8]:
def cv_and_display(preprocessor, model, name, results_df, fine_grained=False):
    """
    train model and return the cross validation results

    preprocessor: (sklearn ColumnTransformer) sklearn object for feature transformation
    model: (sklearn Classifier) initialized sklearn classifier
    name: (str) a name that is shown when the result is displayed
    results_df: (dict) the dictionary to store cross validation results
    fine_grained: (str) {True, False} True: model trains with 3 class classification instead of 2. Default is False

    return: (dict) results_df
    """
    pipeline = make_pipeline(preprocessor, model)
    if fine_grained:
        y_t = y_train
        y_v = y_val
    else:
        y_t = y_train_binary
        y_v = y_val_binary

    scores = cross_validate(pipeline, X_train, y_t, X_val, y_v)

    results_df[name] = scores

    return results_df

## Forward feature selection - Greedy
Forward feature selection (greedy) is the feature selection process accomplished through sklearn's `SequentialFeatureSelector` function. It determines the top k features (k is a variable) using feature importance determined by the model

In [9]:
def perform_sfs_cv_and_display(
    preprocessor,
    model,
    name,
    results_df,
    n_features,
    direction,
    fine_grained=False,
):
    """
    generate a feature selection pipeline for svm models and perform cross validation

    preprocessor: (sklearn ColumnTransformer) sklearn object for feature transformation
    model: (sklearn Classifier) initialized sklearn classifier
    name: (str) a name that is shown when the result is displayed
    results_df: (dict) the dictionary to store cross validation results
    n_features: (int) argument passed into the `n_features_to_select` argument in SequentialFeatureSelector
    direction: (str) {'forward', 'backward'}, argument passe dinto the direction argument in SequentialFeatureSelector
    fine_grained: (str) {True, False} True: model trains with 3 class classification instead of 2. Default is False

    return: (dict) results_df
    """
    # initialize selector
    sfs = SequentialFeatureSelector(
        model,
        n_features_to_select=n_features,
        scoring="accuracy",
        direction=direction,
        cv=2,
        n_jobs=-1,
    )
    sfs_pipeline = make_pipeline(preprocessor, sfs, model)

    if fine_grained:
        y_t = y_train
        y_v = y_val
    else:
        y_t = y_train_binary
        y_v = y_val_binary

    # fit
    sfs_pipeline.fit(X_train, y_t)

    # features selected
    feats_selected = list(compress(X_train.columns, sfs_pipeline[1].get_support()))
#     print(f"features selected:\n{feats_selected}")

    # subset data
    cv_X_train = X_train[feats_selected]
    cv_X_val = X_val[feats_selected]

    # cv with selected features
    cv_preprocessor = make_column_transformer((StandardScaler(), feats_selected))
    pipeline = make_pipeline(cv_preprocessor, model)

    scores = cross_validate(pipeline, cv_X_train, y_t, cv_X_val, y_v)

    results_df[f"{name} + {n_features} features"] = scores

    return results_df, feats_selected

### Binary Classification

In [10]:
binary_results = {}

preprocessor = make_column_transformer((StandardScaler(), feat_names))

classifiers = {
    "DecisionTree": DecisionTreeClassifier(random_state=123, max_depth=8),
    "RandomForest": RandomForestClassifier(random_state=123),
    "LightGBM": LGBMClassifier(random_state=123),
    "XGBoost": XGBClassifier(random_state=123),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=123),
}

for (name, model) in classifiers.items():
    if name == "DecisionTree":
        for d in range(1, 9):
            model = DecisionTreeClassifier(random_state=123, max_depth=d)
            _name = f"{name} + depth {d}"

            results_df = cv_and_display(
                preprocessor, model, _name, binary_results, fine_grained=False
            )
    else:
        results_df = cv_and_display(
            preprocessor, model, name, binary_results, fine_grained=False
        )

display(pd.DataFrame(binary_results))

C:\Users\rsss9\miniconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:51:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,DecisionTree + depth 1,DecisionTree + depth 2,DecisionTree + depth 3,DecisionTree + depth 4,DecisionTree + depth 5,DecisionTree + depth 6,DecisionTree + depth 7,DecisionTree + depth 8,RandomForest,LightGBM,XGBoost,CatBoost
Training_score,0.836576,0.863813,0.887160,0.922179,0.953307,0.984436,0.992218,0.996109,1.00000,1.000000,1.000000,1.000000
Validation_score,0.843750,0.875000,0.875000,0.750000,0.750000,0.812500,0.812500,0.750000,0.81250,0.843750,0.812500,0.843750
Training_time,0.021940,0.021941,0.032911,0.013963,0.018950,0.015956,0.017950,0.027924,0.39893,0.199467,0.662740,13.329520
Prediction_time,0.010970,0.012966,0.013963,0.006980,0.004988,0.005983,0.077793,0.035908,0.02992,0.013962,0.013962,0.012964


In [11]:
# loop through models and number of features to find the best combination
classifiers = {
    "DecisionTree": DecisionTreeClassifier(random_state=123, max_depth=8),
    "RandomForest": RandomForestClassifier(random_state=123),
    "LightGBM": LGBMClassifier(random_state=123),
}

binary_feats = defaultdict(list)
for (name, model) in classifiers.items():
    for i in tqdm(
        range(1, len(feat_names)), desc=f"{name} Feature Selection", dynamic_ncols=True
    ):
        if name == "DecisionTree":
            for d in range(1, 9):
                model = DecisionTreeClassifier(random_state=123, max_depth=d)
                _name = f"{name} + depth {d}"

                results_df, feats = perform_sfs_cv_and_display(
                    preprocessor,
                    model,
                    _name,
                    binary_results,
                    n_features=i,
                    direction="forward",
                    fine_grained=False,
                )

                binary_feats[name].append(feats)
        else:
            results_df, feats = perform_sfs_cv_and_display(
                preprocessor,
                model,
                name,
                binary_results,
                n_features=i,
                direction="forward",
                fine_grained=False,
            )

            binary_feats[name].append(feats)

LightGBM Feature Selection: 100%|██████████████████████████████████████████| 43/43 [12:42<00:00, 17.72s/it]


In [12]:
# sort by Validation_score and display
display(pd.DataFrame(binary_results).sort_values("Validation_score", 1, ascending=False))

,DecisionTree + depth 8 + 4 features,DecisionTree + depth 8 + 3 features,DecisionTree + depth 4 + 2 features,DecisionTree + depth 4 + 4 features,DecisionTree + depth 8 + 30 features,DecisionTree + depth 4 + 3 features,DecisionTree + depth 8 + 37 features,DecisionTree + depth 7 + 24 features,RandomForest + 10 features,DecisionTree + depth 2 + 39 features,...,DecisionTree + depth 8 + 20 features,DecisionTree + depth 6 + 13 features,DecisionTree + depth 7 + 35 features,DecisionTree + depth 8 + 18 features,DecisionTree + depth 5 + 15 features,DecisionTree + depth 8 + 17 features,DecisionTree + depth 5 + 17 features,DecisionTree + depth 5 + 19 features,DecisionTree + depth 6 + 14 features,DecisionTree + depth 5 + 40 features
Training_score,0.972763,0.937743,0.887160,0.898833,0.976654,0.891051,1.000000,0.992218,1.000000,0.863813,...,0.992218,0.976654,0.996109,0.996109,0.949416,0.996109,0.949416,0.953307,0.976654,0.957198
Validation_score,0.937500,0.937500,0.906250,0.906250,0.906250,0.906250,0.906250,0.906250,0.906250,0.875000,...,0.718750,0.718750,0.718750,0.718750,0.718750,0.718750,0.687500,0.687500,0.687500,0.687500
Training_time,0.009974,0.008984,0.009975,0.002156,0.009073,0.010971,0.014960,0.008975,0.185011,0.007977,...,0.007977,0.007978,0.020943,0.007978,0.010971,0.011968,0.011966,0.009974,0.006980,0.010970
Prediction_time,0.005983,0.005984,0.004987,0.009008,0.004088,0.008976,0.004987,0.002992,0.014536,0.003989,...,0.003989,0.003025,0.008978,0.002992,0.004987,0.002992,0.002991,0.003989,0.002992,0.005986


In [47]:
print("Best model: Decision Tree of depth 8 with 4 features")
print(f"Selected features:\n{binary_feats['DecisionTree'][4*8-1]}")

Best model: Decision Tree of depth 8 with 4 features
Selected features:
['avg_sent_length', 'syllables_per_sentence', 'avg_parse_tree_depth', 'SYM']


#### Evaluate best model on test data

In [52]:
selected_features = [
    "avg_sent_length",
    "syllables_per_sentence",
    "avg_parse_tree_depth",
    "SYM",
]
X_train_selected = X_train[selected_features]  # Select train data
X_test_selected = X_test[selected_features]  # Select test data
best_model = DecisionTreeClassifier(random_state=123, max_depth=8)
# Train the model
best_model.fit(X_train_selected, y_train_binary)

predicted_y_test_binary = best_model.predict(X_test_selected)
print("Test accuracy %0.3f" % (accuracy_score(y_test_binary, predicted_y_test_binary)))
print("Test Set Classification Report:")
print(classification_report(y_test_binary, predicted_y_test_binary))

Test accuracy 0.844
Test Set Classification Report:
              precision    recall  f1-score   support

           A       0.93      0.76      0.84        17
           B       0.78      0.93      0.85        15

    accuracy                           0.84        32
   macro avg       0.85      0.85      0.84        32
weighted avg       0.86      0.84      0.84        32



### 3-Class Classification

In [19]:
multiclass_results = {}

preprocessor = make_column_transformer((StandardScaler(), feat_names))

classifiers = {
    "DecisionTree": DecisionTreeClassifier(random_state=123, max_depth=8),
    "RandomForest": RandomForestClassifier(random_state=123),
    "LightGBM": LGBMClassifier(random_state=123),
    "XGBoost": XGBClassifier(random_state=123),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=123),
}

for (name, model) in classifiers.items():
    if name == "DecisionTree":
        for d in range(1, 9):
            model = DecisionTreeClassifier(random_state=123, max_depth=d)
            _name = f"{name} + depth {d}"

            results_df = cv_and_display(
                preprocessor, model, _name, multiclass_results, fine_grained=True
            )
    else:
        results_df = cv_and_display(
            preprocessor, model, name, multiclass_results, fine_grained=True
        )

display(pd.DataFrame(multiclass_results))

C:\Users\rsss9\miniconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:46:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,DecisionTree + depth 1,DecisionTree + depth 2,DecisionTree + depth 3,DecisionTree + depth 4,DecisionTree + depth 5,DecisionTree + depth 6,DecisionTree + depth 7,DecisionTree + depth 8,RandomForest,LightGBM,XGBoost,CatBoost
Training_score,0.715953,0.754864,0.785992,0.859922,0.906615,0.964981,0.988327,1.000000,1.000000,1.000000,1.000000,1.000000
Validation_score,0.750000,0.750000,0.750000,0.718750,0.656250,0.687500,0.656250,0.656250,0.750000,0.687500,0.625000,0.750000
Training_time,0.014959,0.014957,0.029929,0.033908,0.019946,0.011969,0.011967,0.016954,0.266291,0.259305,0.394455,20.845083
Prediction_time,0.005987,0.008975,0.012967,0.006980,0.002991,0.004987,0.003991,0.002991,0.014961,0.006980,0.013963,0.006981


In [20]:
# loop through models and number of features to find the best combination
classifiers = {
    "DecisionTree": DecisionTreeClassifier(random_state=123, max_depth=8),
    "RandomForest": RandomForestClassifier(random_state=123),
    "LightGBM": LGBMClassifier(random_state=123),
}

multiclass_feats = defaultdict(list)
for (name, model) in classifiers.items():
    for i in tqdm(
        range(1, len(feat_names)), desc=f"{name} Feature Selection", dynamic_ncols=True
    ):
        if name == "DecisionTree":
            for d in range(1, 9):
                model = DecisionTreeClassifier(random_state=123, max_depth=d)
                _name = f"{name} + depth {d}"

                results_df, feats = perform_sfs_cv_and_display(
                    preprocessor,
                    model,
                    _name,
                    multiclass_results,
                    n_features=i,
                    direction="forward",
                    fine_grained=True,
                )

                multiclass_feats[name].append(feats)
        else:
            results_df, feats = perform_sfs_cv_and_display(
                preprocessor,
                model,
                name,
                multiclass_results,
                n_features=i,
                direction="forward",
                fine_grained=True,
            )

            multiclass_feats[name].append(feats)

LightGBM Feature Selection: 100%|██████████████████████████████████████████| 43/43 [28:25<00:00, 39.67s/it]


In [21]:
# sort by Validation_score and display
display(pd.DataFrame(multiclass_results).sort_values("Validation_score", 1, ascending=False))

,LightGBM + 33 features,LightGBM + 38 features,LightGBM + 37 features,LightGBM + 35 features,LightGBM + 24 features,LightGBM + 29 features,DecisionTree + depth 4 + 5 features,DecisionTree + depth 5 + 26 features,DecisionTree + depth 4 + 6 features,DecisionTree + depth 4 + 13 features,...,DecisionTree + depth 8 + 14 features,DecisionTree + depth 7 + 17 features,DecisionTree + depth 7 + 11 features,DecisionTree + depth 8 + 26 features,DecisionTree + depth 8 + 27 features,DecisionTree + depth 6 + 3 features,DecisionTree + depth 7 + 18 features,DecisionTree + depth 7 + 10 features,DecisionTree + depth 7 + 9 features,DecisionTree + depth 8 + 23 features
Training_score,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,0.832685,0.875486,0.832685,0.832685,...,0.968872,0.972763,0.968872,1.000000,1.000000,0.883268,0.972763,0.968872,0.968872,1.000000
Validation_score,0.812500,0.812500,0.812500,0.812500,0.81250,0.812500,0.781250,0.781250,0.781250,0.781250,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.468750
Training_time,0.186332,0.171256,0.166169,0.168021,0.14900,0.161067,0.006981,0.009973,0.004987,0.006981,...,0.007978,0.007978,0.006981,0.010939,0.013962,0.006984,0.007979,0.006982,0.006981,0.012966
Prediction_time,0.009070,0.009066,0.009100,0.008977,0.00907,0.008134,0.002992,0.003989,0.003991,0.002992,...,0.002992,0.003991,0.003990,0.003989,0.004990,0.002027,0.002992,0.002992,0.002992,0.005980


In [48]:
print("Best model: LightGBM with 24 features")
print(f"Selected features:\n{multiclass_feats['LightGBM'][24-1]}")

Best model: LightGBM with 24 features
Selected features:
['avg_sent_length', 'pronoun_density', 'fernandez_huerta_score', 'syllables_per_sentence', 'avg_degree_of_abstraction', 'min_degree_of_abstraction', 'avg_ambiguation_all_words', 'avg_ambiguation_content_words', 'avg_parse_tree_depth', 'Past', 'AUX', 'CONJ', 'DET', 'NOUN', 'NUM', 'PART', 'PRON', 'PUNCT', 'SYM', 'VERB', 'X', 'EOL', 'SPACE', 'FUNCTION']


In [53]:
selected_features = [
    "avg_sent_length",
    "pronoun_density",
    "fernandez_huerta_score",
    "syllables_per_sentence",
    "avg_degree_of_abstraction",
    "min_degree_of_abstraction",
    "avg_ambiguation_all_words",
    "avg_ambiguation_content_words",
    "avg_parse_tree_depth",
    "Past",
    "AUX",
    "CONJ",
    "DET",
    "NOUN",
    "NUM",
    "PART",
    "PRON",
    "PUNCT",
    "SYM",
    "VERB",
    "X",
    "EOL",
    "SPACE",
    "FUNCTION",
]
X_train_selected = X_train[selected_features]  # Select train data
X_test_selected = X_test[selected_features]  # Select test data
best_model = LGBMClassifier(random_state=123)
# Train the model
best_model.fit(X_train_selected, y_train)

predicted_y_test = best_model.predict(X_test_selected)
print("Test accuracy %0.3f" % (accuracy_score(y_test, predicted_y_test)))
print("Test Set Classification Report:")
print(classification_report(y_test, predicted_y_test))

Test accuracy 0.625
Test Set Classification Report:
              precision    recall  f1-score   support

          A1       0.62      0.45      0.53        11
          A2       0.29      0.33      0.31         6
           B       0.76      0.87      0.81        15

    accuracy                           0.62        32
   macro avg       0.56      0.55      0.55        32
weighted avg       0.63      0.62      0.62        32

